In [1]:
pip install anthropic --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
from anthropic import Anthropic
import re

client = Anthropic(
    api_key="",
)
MODEL_NAME = "claude-3-opus-20240229"


In [3]:
def toolreader(name, description, parameters):
    constructed_prompt = (
        "<tool_description>\n"
        f"<tool_name>{name}</tool_name>\n"
        "<description>\n"
        f"{description}\n"
        "</description>\n"
        "<parameters>\n"
        + "\n".join(f"<parameter>\n<name>{parameter['name']}</name>\n<type>{parameter['type']}</type>\n<description>{parameter['description']}</description>\n</parameter>" for parameter in parameters) +
        "\n</parameters>\n"
        "</tool_description>"
    )
    return constructed_prompt


def agent(system, tools):
    agentsystemplustools = (
        system +
        "\n\n"
        "In this environment you have access to a set of tools you can use to answer the user's question.\n"
        "\n"
        "You may call them like this:\n"
        "<function_calls>\n"
        "<invoke>\n"
        "<tool_name>$TOOL_NAME</tool_name>\n"
        "<parameters>\n"
        "<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>\n"
        "...\n"
        "</parameters>\n"
        "</invoke>\n"
        "</function_calls>\n"
        "\n"
        "Here are the tools available:\n"
        "<tools>\n"
        + '\n'.join([tool for tool in tools]) +
        "\n</tools>"
    )
    return agentsystemplustools

def wordboxprocess(agent,message):
    messages = [
      {
        "role": "user",
        "content": message
      }
    ]
    completion = client.messages.create(
      model=MODEL_NAME,
      max_tokens=1024,
      messages=messages,
      system=agent,
      stop_sequences=["\n\nHuman:", "\n\nAssistant", "</function_calls>"]
    ).content[0].text

    return completion


In [4]:
tool_name = "calculator"
tool_description = """Calculator function for doing basic arithmetic. 
Supports addition, subtraction, multiplication"""
parameters = [
    {
        "name": "first_operand",
        "type": "int",
        "description": "First operand (before the operator)"
    },
    {
        "name": "second_operand",
        "type": "int",
        "description": "Second operand (after the operator)"
    },
    {
        "name": "operator",
        "type": "str",
        "description": "The operation to perform. Must be either +, -, *, or /"
    }
]



system = "you are bad ass agent"

multiplication_message = {
    "role": "user", 
    "content": "Multiply 1,984,135 by 9,343,116"
}

xmas = "Multiply 1,984,135 by 9,343,116"

In [5]:
tool1 = toolreader(tool_name, tool_description, parameters)
agent1 = agent(system,[tool1])
from tools.tuesday import Tool

agent_uid='agent_k'
tool = Tool(agent_uid)
resultC = wordboxprocess(agent1,xmas)
print(resultC)

ModuleNotFoundError: No module named 'tools'

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

In [ ]:
def extract_between_tags(tag: str, string: str, strip: bool = False) -> list[str]:
    ext_list = re.findall(f"<{tag}>(.+?)</{tag}>", string, re.DOTALL)
    if strip:
        ext_list = [e.strip() for e in ext_list]
    return ext_list


first_operand = int(extract_between_tags("first_operand", resultC)[0])
second_operand = int(extract_between_tags("second_operand", resultC)[0])
operator = extract_between_tags("operator", resultC)[0]

result = do_pairwise_arithmetic(first_operand, second_operand, operator)
print(f"{result:,}")

In [ ]:
def resultoutputprocp(results: list[dict]):
    f_results = [{
        'tool_name': 'do_pairwise_arithmetic',
        'tool_result': result
    }]
    
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>" 
            for res in f_results
        ) + "\n</function_results>"
    )
    
    return constructed_prompt



function_results = resultoutputprocp(result)
print(function_results)


In [ ]:
partial_assistant_message = resultC + "</function_calls>" + function_results
final_message = client.messages.create(
    model=MODEL_NAME,
    max_tokens=2000,
    messages=[
        multiplication_message,
        {
            "role": "assistant",
            "content": partial_assistant_message
        }
    ],
    system=agent1,
).content[0].text
print(partial_assistant_message + final_message)